In [54]:
import joblib
import re
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
#import streamlit as st
import simplejson as json
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from tensorflow import keras
from keras_preprocessing.sequence import pad_sequences
import pickle
import pandas as pd
import numpy as np

MAX_SEQUENCE_LENGTH = 1000
# This is fixed.
EMBEDDING_DIM = 200
roc_t = 0.36430258

with open('C:/Users/falco/Desktop/directory/wildlife_syndrome_app/models/keras_tokenizer_v3_08_21_2022.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = keras.models.load_model('C:/Users/falco/Desktop/directory/wildlife_syndrome_app/models/model_v3_08_21_2022.h5')


#st.write("# Wildlife Clinical Classification App")

#message_text = st.text_input("Enter text describing clincal evaluation, reason/s for admission")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
labels = ['Clinically healthy', 'Dermatologic disease', 'Gastrointestinal disease', 'Hematologic disease', 'Neurologic disease', 'Nonspecific', 'Nutritional disease', 'Ocular disease', 'Physical injury', 'Respiratory disease', 'Urogenital disease']





In [39]:
new_case = "injured, flea ridden, flea, dirt, pale"

In [40]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

nc = clean_text(text = new_case)
nc

'injured flea ridden flea dirt pale'

In [49]:
new_case = [nc]
seq = tokenizer.texts_to_sequences(new_case)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
print(pred)
sorted_pred = sorted(pred[0])
print(sorted_pred)
print(labels[np.argmax(pred)])
print(np.max(pred))

1/1 [==============================] - 0s 65ms/step
[[4.0399078e-02 3.8605982e-03 6.5266655e-04 2.2354666e-02 4.2565293e-03
  8.1080467e-02 5.4761614e-03 2.1628998e-04 8.3666742e-01 4.6858541e-03
  3.5030671e-04]]
[0.00021628998, 0.0003503067, 0.00065266655, 0.0038605982, 0.0042565293, 0.004685854, 0.0054761614, 0.022354666, 0.04039908, 0.08108047, 0.8366674]
Physical injury
0.8366674


In [51]:
labels[np.argmax(pred)], np.max(pred)

('Physical injury', 0.8366674)

In [60]:
def predict_conditions(new_case):
    new_case = [new_case]
    #print(new_case)
    seq = tokenizer.texts_to_sequences(new_case)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    sorted_pred = sorted(pred[0])
    if sorted_pred[-1]-sorted_pred[-2] >= roc_t:
        #print ("Final Clinical Classifiaction Prediction")
        #print(labels[np.argmax(pred)])
        clinical_class_1 = labels[np.argmax(pred)] 
        clinical_class_2 = None
    else:
        #print("First Prediction")
        #print(labels[np.argmax(pred)])
        #print("")
        #print("Second Prediction")
        #print(labels[list(pred[0]).index(sorted_pred[-2])])
        clinical_class_1 = labels[np.argmax(pred)]
        clinical_class_2 = labels[list(pred[0]).index(sorted_pred[-2])]
    return clinical_class_1, clinical_class_2

In [61]:
%%time
predict_conditions(nc)

1/1 [==============================] - 0s 67ms/step
CPU times: total: 109 ms
Wall time: 115 ms


('Physical injury', None)

In [62]:
def preprocess_and_predict_input_data(new_case_description):
    nc = clean_text(text = new_case_description)
    cc1, cc2 = predict_conditions(nc)
    return cc1, cc2
    